In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [13]:
from keras.datasets import mnist
import sklearn.preprocessing
import sklearn.metrics

In [14]:
# Преобразование данных: нормализация и преобразование в тензоры
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Нормализация данных
])

# Загрузка данных
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Создание DataLoader для удобной работы с данными
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [15]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()

train_x = train_x / 255
test_x = test_x / 255

train_x.resize(60000, 784)
test_x.resize(10000, 784)

encoder = sklearn.preprocessing.OneHotEncoder()
train_y = encoder.fit_transform(train_y[..., None]).toarray()
test_y = encoder.fit_transform(test_y[..., None]).toarray()

train_x.shape, train_y.shape

((60000, 784), (60000, 10))

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Свёрточные слои
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # Слои подвыборки
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # Полносвязные слои
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Преобразование в 1D тензор
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Создание модели
model = SimpleCNN()

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# Количество эпох
num_epochs = 5

# Обучение модели
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # Обнуление градиентов
        optimizer.zero_grad()
        
        # Прямой проход
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Эпоха [{epoch+1}/{num_epochs}], Потери: {running_loss/len(train_loader):.4f}')

Эпоха [1/5], Потери: 0.1531
Эпоха [2/5], Потери: 0.0439
Эпоха [3/5], Потери: 0.0292
Эпоха [4/5], Потери: 0.0224
Эпоха [5/5], Потери: 0.0162


In [7]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Точность модели на тестовых данных: {100 * correct / total:.2f}%')

Точность модели на тестовых данных: 98.88%


In [21]:
model(torch.tensor(train_x[0], dtype=torch.float32).reshape((1, 28, 28)))

tensor([[-5.2315, -4.1108, -4.9194,  9.0929, -6.4677,  7.0808, -4.6419, -4.6903,
          0.2393,  0.7539]], grad_fn=<AddmmBackward0>)